In [2]:
#1、病理切片二值化
import cv2
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import glob, os
files = "pic_initial/20.jpg"
img = cv2.imread(files, 0)
ret2, mask_OTSU = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
kernel_size = (1,1) #可调参
img = cv2.blur(mask_OTSU, kernel_size)
ret2, mask_OTSU = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
img = cv2.blur(mask_OTSU, kernel_size)
ret2, mask_OTSU = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
#设置卷积核
kernel = np.ones((1,1), np.uint8) #可调参
dst = cv2.morphologyEx(mask_OTSU, cv2.MORPH_OPEN, kernel)
cv2.imwrite("pic_over/20.jpg",dst)
#绘制查看效果
'''
plt.figure(figsize=(8, 6))
plt.imshow(dst,cmap='gray')
plt.axis("off")
cv2.imwrite("pic_over/"+str(i)+".jpg",dst)
plt.show()
'''

'\nplt.figure(figsize=(8, 6))\nplt.imshow(dst,cmap=\'gray\')\nplt.axis("off")\ncv2.imwrite("pic_over/"+str(i)+".jpg",dst)\nplt.show()\n'

In [4]:
#2、二值图调整大小并读取
from PIL import Image
import glob, os

size = [650, 650]
file, ext = os.path.splitext(files)
im = Image.open(files)
im.thumbnail(size)
im.save(file + ".jpg", "JPEG")
dst = cv2.imread(files, 0)

In [7]:
#定义旋转操作
def rotate(image, angle, center=None, scale=1.0):
    # grab the dimensions of the image
    (h, w) = image.shape[:2]

    # if the center is None, initialize it as the center of
    # the image
    if center is None:
        center = (w // 2, h // 2)

    # perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    # return the rotated image
    return rotated

In [8]:
#3、分别获得旋转90°、180°、270°的二值图
dst1=rotate(dst,90)
dst2=rotate(dst,180)
dst3=rotate(dst,270)

In [9]:
#4、将二值图随机进行拼接
dst_ = np.concatenate((dst1,dst1,dst3,dst,dst2,dst3,dst,dst1,dst2,dst1,dst,dst3), axis=1)
dst_2 = np.concatenate((dst1,dst3,dst,dst,dst3,dst3,dst2,dst2,dst,dst,dst,dst1), axis=1)
dst_3 = np.concatenate((dst2,dst,dst1,dst2,dst2,dst1,dst2,dst1,dst2,dst,dst,dst2), axis=1)
dst_4= np.concatenate((dst2,dst3,dst,dst,dst3,dst3,dst,dst,dst,dst,dst,dst2), axis=1)
dst_5= np.concatenate((dst,dst2,dst2,dst1,dst3,dst3,dst,dst,dst,dst,dst,dst3), axis=1)
dst__= np.concatenate((dst_,dst_2,dst_3,dst_4,dst_5), axis=0)
dst6=np.transpose(dst__)
#绘制查看效果
'''
plt.imshow(dst6,cmap='gray')
plt.axis("off")
plt.show()
'''

In [17]:
#5、将二值图0，255转换为0，1，并翻转颜色。
dst6 = dst6.astype(np.double) / 255
where_0 = np.where(dst6 == 0)
where_1 = np.where(dst6 == 1)
dst6[where_0] = 1
dst6[where_1] = 0

In [20]:
#6、设置参数
num=250  #通过Parameter extraction获得的肺泡个数
dx=num*12*5/1500/len(dst[0])/len(dst)/10000

In [22]:
#7、获得 mat格式的模体
from scipy.io import savemat
a_dict = {'phantom': dst6, 'dx': np.sqrt(dx)}
savemat('E001_disturbed.mat', {'phant': a_dict})